In [1]:
import re
import torch
import torch.nn.functional as F
from lib.utils import load, preprocess, get_dict, ask_dict, sanitize

In [2]:
NGRAM_CONTEXT_WINDOW = 2

END_TOKEN = "[end]"
WITH_END_TOKEN = True

## Preprocessing

In [3]:
list_str = load('./data/songs.json')
del list_str["rows"][49] # Shape of You
del list_str["rows"][50 - 1] # You Need Me, I Don't Need You
del list_str["rows"][70 - 2] # Dive
del list_str["rows"][76 - 3] # Take Me Back to London

list_str = [preprocess(str["row"]["text"]) for str in list_str["rows"] if str["row"]["text"]]

if WITH_END_TOKEN:
    # Treat new lines as a word or not
    list_str = [str.replace("\n", f" {END_TOKEN} ") for str in list_str]

list_str

['The club isnt the best place to find a lover [end] So the bar is where I go [end] Me and my friends at the table doing shots [end] Drinking fast and then we talk slow [end] And you come over and start up a conversation with just me [end] And trust me Ill give it a chance now [end] Take my hand stop put Van the Man on the jukebox [end] And then we start to dance and now Im singing like [end] Girl you know I want your love [end] Your love was handmade for somebody like me [end] Come on now follow my lead [end] I may be crazy dont mind me [end] Say boy lets not talk too much [end] Grab on my waist and put that body on me [end] Come on now follow my lead [end] Come come on now follow my lead [end] Im in love with the shape of you [end] We push and pull like a magnet do [end] Although my heart is falling too [end] Im in love with your body [end] And last night you were in my room [end] And now my bed sheets smell like you [end] Every day discovering something brand new [end] Im in love wi

## Intializations

In [4]:
# Split the lines into words and lowercase them
list_list_words = sanitize(list_str)

list_list_words

[['the',
  'club',
  'isnt',
  'the',
  'best',
  'place',
  'to',
  'find',
  'a',
  'lover',
  '[end]',
  'so',
  'the',
  'bar',
  'is',
  'where',
  'i',
  'go',
  '[end]',
  'me',
  'and',
  'my',
  'friends',
  'at',
  'the',
  'table',
  'doing',
  'shots',
  '[end]',
  'drinking',
  'fast',
  'and',
  'then',
  'we',
  'talk',
  'slow',
  '[end]',
  'and',
  'you',
  'come',
  'over',
  'and',
  'start',
  'up',
  'a',
  'conversation',
  'with',
  'just',
  'me',
  '[end]',
  'and',
  'trust',
  'me',
  'ill',
  'give',
  'it',
  'a',
  'chance',
  'now',
  '[end]',
  'take',
  'my',
  'hand',
  'stop',
  'put',
  'van',
  'the',
  'man',
  'on',
  'the',
  'jukebox',
  '[end]',
  'and',
  'then',
  'we',
  'start',
  'to',
  'dance',
  'and',
  'now',
  'im',
  'singing',
  'like',
  '[end]',
  'girl',
  'you',
  'know',
  'i',
  'want',
  'your',
  'love',
  '[end]',
  'your',
  'love',
  'was',
  'handmade',
  'for',
  'somebody',
  'like',
  'me',
  '[end]',
  'come',
  'o

## Dictionary

In [5]:
dict_key_tuple = get_dict(list_list_words, NGRAM_CONTEXT_WINDOW)
list_key_tuple_answers = ask_dict(dict_key_tuple, "shape")

list_key_tuple = [(key, tuple[0], tuple[1]) for key, tuple in dict_key_tuple.items()]
tensor_probs = torch.tensor([tuple[1] for _, tuple in dict_key_tuple.items()], dtype=torch.float32)

def bigram(tensor_probs, list_key_tuple):
    index = torch.multinomial(tensor_probs, num_samples=1, replacement=True).item()
    return list_key_tuple[index][0]

sorted(list_key_tuple_answers, key=lambda x: x[1], reverse=True)

[('shape+of', 9, 0.00019097332739194094, 1.0)]